In [5]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from datetime import datetime
from datetime import timedelta 
import time
import numpy as np

## 先連好資料庫

In [6]:
#這是雷射機台的東西

try:
    cnx = mysql.connector.connect(host='',
                                  user='',
                                  passwd='',
                                  database='svfs',
                                  )

    if cnx.is_connected():
        # 顯示資料庫版本
        db_Info = cnx.get_server_info()
        cursor = cnx.cursor(named_tuple=True)
        print('svfs database version:',db_Info)

except Error as e:
    print("資料庫連接失敗：", e)


sql = "SELECT machine , part_name , result_start_time , result_end_time , TIMEDIFF(result_end_time,result_start_time) as `delta_time` , result_count , result_badcount \
       FROM parts_log\
       WHERE result_start_time >= '2020-05-31 00:00:00' and result_end_time <='2020-08-01 23:59:59' ORDER BY result_start_time ASC"



#=============================================================#
#這邊是BOM表 從子件去看所屬母件

try:
    cnx_bom = mysql.connector.connect(host='59.120.87.160',
                                  user='smb',
                                  passwd='Csie3621',
                                  database='smbsource',
                                  )

    if cnx_bom.is_connected():
        # 顯示資料庫版本
        db_Info = cnx_bom.get_server_info()
        cursor_bom = cnx_bom.cursor(named_tuple=True)
        print('smbsource database version:',db_Info)

except Error as e:
    print("資料庫連接失敗：", e)

sql_bom = "SELECT product_id,parent_id,amount \
           FROM bom\
           WHERE mp_id = 'MC' " #   and product_id = %s

svfs database version: 5.7.29-32
smbsource database version: 5.7.29-32


In [7]:
cursor.execute(sql)
parts_log = cursor.fetchall()  #先獲取折床報表DATA

## 這邊是處理它的對應週數、週總耗時

In [8]:
week_number=23 #從哪一周開始算
count_time = timedelta()  #耗時加總用
index_list = [] #把這一週的東西新增進去

real_parts_log=[]

for number,index in enumerate(parts_log):

    if week_number == int(index.result_start_time.strftime("%U"))+1:  #第一次進的會是23
        count_time += index[4] #裡面的所有耗時加總
        index_list.append(index) #把屬於23週的東西新增到List

    elif week_number != int(index.result_start_time.strftime("%U"))+1: #當它不再=23，應該說當他換週時:
        for index_save in index_list: #剛剛新增進去list裡面的東西
            real_parts_log.append([index_save[0],
                                    index_save[1],
                                    int(index_save.result_start_time.strftime("%U"))+1,
                                    count_time,
                                    index_save[2],
                                    index_save[3],
                                    index_save[4],
                                    index_save[5],
                                    index_save[6],
                                    ]) #list裡面的東西
            
        count_time = timedelta() #重畳
        index_list = [] #重畳
        week_number = int(index.result_start_time.strftime("%U"))+1 #因為跳到下一週了，它本身會進入elif，這邊的週數也變成了那一週
        
        count_time += index[4] #裡面的所有耗時加總
        index_list.append(index) #把屬於下一週的東西新增到List
        
    if number==len(parts_log)-1: #當它來到最後一行時
        for index_save in index_list:
            real_parts_log.append([index_save[0],
                                    index_save[1],
                                    int(index_save.result_start_time.strftime("%U"))+1,
                                    count_time,
                                    index_save[2],
                                    index_save[3],
                                    index_save[4],
                                    index_save[5],
                                    index_save[6],
                                    ])#list裡面的東西

## 這邊是用  parts_log折床報表中的  子件  把  母件  撈出來再合在一起

##### 1 比較快的方法，一次從資料庫撈全部，然後再用pd merge，減少每次for迴圈都跑一次sql的東西

In [9]:
cursor_bom.execute(sql_bom)  
data_bom = cursor_bom.fetchall()
for i ,q in enumerate(data_bom):
    print(i,q)

0 Row(product_id='JU2020030201-B', parent_id='JU2020030201', amount=3)
1 Row(product_id='JU2020030201-C', parent_id='JU2020030201', amount=1)
2 Row(product_id='DA23568A-A', parent_id='DA23568A', amount=1)
3 Row(product_id='DA23568A-B', parent_id='DA23568A', amount=1)
4 Row(product_id='DA23567A-A', parent_id='DA23567A', amount=1)
5 Row(product_id='1VK2A2003B1-A', parent_id='1VK2A2003B1', amount=1)
6 Row(product_id='2SLAE1020B1-A', parent_id='2SLAE1020B1', amount=1)
7 Row(product_id='1QA3F1013A0-A', parent_id='1QA3F1013A0', amount=1)
8 Row(product_id='1QA3F1013A0-A1', parent_id='1QA3F1013A0', amount=1)
9 Row(product_id='1QA3F1013A0-B', parent_id='1QA3F1013A0', amount=1)
10 Row(product_id='1QA3F1013A0-C', parent_id='1QA3F1013A0', amount=1)
11 Row(product_id='1QA3F1013A0-D', parent_id='1QA3F1013A0', amount=1)
12 Row(product_id='1QA3F1013A0-E', parent_id='1QA3F1013A0', amount=1)
13 Row(product_id='1QA3F1013A0-F', parent_id='1QA3F1013A0', amount=1)
14 Row(product_id='1QA3F1013A0-G', parent_i

1040 Row(product_id='PT750FU0171-E', parent_id='PT750FU0171', amount=1)
1041 Row(product_id='PT750FU0620-A', parent_id='PT750FU0620', amount=1)
1042 Row(product_id='PT750FU0314-A', parent_id='PT750FU0314', amount=1)
1043 Row(product_id='PT750FU0314-E', parent_id='PT750FU0314', amount=1)
1044 Row(product_id='PT550FU0213-A', parent_id='PT550FU0213', amount=1)
1045 Row(product_id='PT750FU0213-A', parent_id='PT750FU0213', amount=1)
1046 Row(product_id='PT750FU0213-C', parent_id='PT750FU0213', amount=1)
1047 Row(product_id='PT750FU0213-D', parent_id='PT750FU0213', amount=2)
1048 Row(product_id='PT750FU0213-E', parent_id='PT750FU0213', amount=2)
1049 Row(product_id='PT750FU0213-F', parent_id='PT750FU0213', amount=1)
1050 Row(product_id='PT750FU0213-G', parent_id='PT750FU0213', amount=1)
1051 Row(product_id='PT750FU0140-A', parent_id='PT750FU0140', amount=1)
1052 Row(product_id='PT750FU0112-A', parent_id='PT750FU0112', amount=1)
1053 Row(product_id='PT750FU0111-A', parent_id='PT750FU0111', am

1795 Row(product_id='1D80D1092C3-A3', parent_id='1D80D1092C3', amount=1)
1796 Row(product_id='1D80D1092C3-B', parent_id='1D80D1092C3', amount=1)
1797 Row(product_id='1D80D1092C3-C', parent_id='1D80D1092C3', amount=1)
1798 Row(product_id='1D80D1092C3-C1', parent_id='1D80D1092C3', amount=1)
1799 Row(product_id='1D80D1092C3-C2', parent_id='1D80D1092C3', amount=1)
1800 Row(product_id='1D80D1092C3-D', parent_id='1D80D1092C3', amount=1)
1801 Row(product_id='1D80D1092C3-D1', parent_id='1D80D1092C3', amount=1)
1802 Row(product_id='1D80D1092C3-E', parent_id='1D80D1092C3', amount=1)
1803 Row(product_id='1D80D1092C3-F', parent_id='1D80D1092C3', amount=1)
1804 Row(product_id='1D80D1092C3-F1', parent_id='1D80D1092C3', amount=1)
1805 Row(product_id='1D80D1092C3-F2', parent_id='1D80D1092C3', amount=1)
1806 Row(product_id='1D80D1092C3-F3', parent_id='1D80D1092C3', amount=1)
1807 Row(product_id='1D80D1092C3-F4', parent_id='1D80D1092C3', amount=1)
1808 Row(product_id='1D80D1092C3-F5', parent_id='1D80D10

2859 Row(product_id='1W50D6015A1-A', parent_id='1W50D6015A1', amount=1)
2860 Row(product_id='1W80D1117A0-A', parent_id='1W80D1117A0', amount=1)
2861 Row(product_id='1W50D3002A1-A', parent_id='1W50D3002A1', amount=1)
2862 Row(product_id='1W50B1316A1-A', parent_id='1W50B1316A1', amount=1)
2863 Row(product_id='1W50D1115A3-A', parent_id='1W50D1115A3', amount=1)
2864 Row(product_id='1W50D1115A3-B', parent_id='1W50D1115A3', amount=1)
2865 Row(product_id='1W50D6007C3-B', parent_id='1W50D6007C3', amount=1)
2866 Row(product_id='1W50D6007C3-D', parent_id='1W50D6007C3', amount=1)
2867 Row(product_id='1W50F2006B0-A', parent_id='1W50F2006B0', amount=1)
2868 Row(product_id='1W50F2006B0-B', parent_id='1W50F2006B0', amount=1)
2869 Row(product_id='1W50F2006B0-C', parent_id='1W50F2006B0', amount=1)
2870 Row(product_id='1W50F2006B0-D', parent_id='1W50F2006B0', amount=2)
2871 Row(product_id='1W50F2006B0-E', parent_id='1W50F2006B0', amount=1)
2872 Row(product_id='1W50D6012B0-A', parent_id='1W50D6012B0', am

4038 Row(product_id='1SL6D1025A1-B', parent_id='1SL6D1025A1', amount=2)
4039 Row(product_id='2SLCD1205A1-A', parent_id='2SLCD1205A1', amount=1)
4040 Row(product_id='1SLCD1273B0-A', parent_id='1SLCD1273B0', amount=1)
4041 Row(product_id='2SLCD1216A0-A', parent_id='2SLCD1216A0', amount=1)
4042 Row(product_id='1SLCD1185A1-A', parent_id='1SLCD1185A1', amount=1)
4043 Row(product_id='2SLCD1007A1-A', parent_id='2SLCD1007A1', amount=1)
4044 Row(product_id='2SLCD1007A1-B', parent_id='2SLCD1007A1', amount=1)
4045 Row(product_id='2SLCD1007A1-C', parent_id='2SLCD1007A1', amount=1)
4046 Row(product_id='2SLCD1007A1-D', parent_id='2SLCD1007A1', amount=1)
4047 Row(product_id='2SLCD1007A1-D1', parent_id='2SLCD1007A1', amount=1)
4048 Row(product_id='2SLCD1007A1-F', parent_id='2SLCD1007A1', amount=1)
4049 Row(product_id='2SLCD1007A1-G', parent_id='2SLCD1007A1', amount=2)
4050 Row(product_id='2SLCD1007A1-I', parent_id='2SLCD1007A1', amount=3)
4051 Row(product_id='2SLCE1202A1-A', parent_id='2SLCE1202A1', a

...

6340 Row(product_id='262090011R01-K', parent_id='262090011R01', amount=1)
6341 Row(product_id='262090011R01-K1', parent_id='262090011R01', amount=1)
6342 Row(product_id='262090011R01-30', parent_id='262090011R01', amount=1)
6343 Row(product_id='262090011R01-31', parent_id='262090011R01', amount=1)
6344 Row(product_id='262090011R01-32', parent_id='262090011R01', amount=1)
6345 Row(product_id='262090011R01-34', parent_id='262090011R01', amount=2)
6346 Row(product_id='262090011R01-90', parent_id='262090011R01', amount=1)
6347 Row(product_id='262090011R01-91', parent_id='262090011R01', amount=1)
6348 Row(product_id='262090011R01-92', parent_id='262090011R01', amount=1)
6349 Row(product_id='262090011R01-93', parent_id='262090011R01', amount=1)
6350 Row(product_id='2DK1G1006A0-A', parent_id='2DK1G1006A0', amount=1)
6351 Row(product_id='2DK1G1006A0-B', parent_id='2DK1G1006A0', amount=2)
6352 Row(product_id='2DK1D1025A0-A', parent_id='2DK1D1025A0', amount=1)
6353 Row(product_id='2DK1D1025A0-C'

7606 Row(product_id='302M002-I1', parent_id='302M002', amount=1)
7607 Row(product_id='302M002-J', parent_id='302M002', amount=1)
7608 Row(product_id='302M002-J1', parent_id='302M002', amount=1)
7609 Row(product_id='302M002-K', parent_id='302M002', amount=1)
7610 Row(product_id='302M002-K1', parent_id='302M002', amount=2)
7611 Row(product_id='302M002-O', parent_id='302M002', amount=1)
7612 Row(product_id='302M002-S', parent_id='302M002', amount=1)
7613 Row(product_id='302M002-T', parent_id='302M002', amount=1)
7614 Row(product_id='302M002-U', parent_id='302M002', amount=1)
7615 Row(product_id='8030102022-A', parent_id='8030102022', amount=1)
7616 Row(product_id='8030102022-B', parent_id='8030102022', amount=1)
7617 Row(product_id='8030102022-C', parent_id='8030102022', amount=1)
7618 Row(product_id='8030102022-D', parent_id='8030102022', amount=2)
7619 Row(product_id='302M025-A', parent_id='302M025', amount=1)
7620 Row(product_id='302M025-B', parent_id='302M025', amount=1)
7621 Row(prod

8297 Row(product_id='3125270760-D', parent_id='3125270760', amount=1)
8298 Row(product_id='3125270760-E', parent_id='3125270760', amount=1)
8299 Row(product_id='CV140407A0-A', parent_id='CV140407A0', amount=1)
8300 Row(product_id='CV140407A0-A1', parent_id='CV140407A0', amount=1)
8301 Row(product_id='CV140407A0-B', parent_id='CV140407A0', amount=1)
8302 Row(product_id='CV140407A0-C', parent_id='CV140407A0', amount=1)
8303 Row(product_id='CV140407A0-D', parent_id='CV140407A0', amount=1)
8304 Row(product_id='CV140407A0-E', parent_id='CV140407A0', amount=1)
8305 Row(product_id='CV140407A0-F', parent_id='CV140407A0', amount=1)
8306 Row(product_id='CV140407A0-G', parent_id='CV140407A0', amount=1)
8307 Row(product_id='CV140407A0-G2', parent_id='CV140407A0', amount=1)
8308 Row(product_id='CV140407A0-H', parent_id='CV140407A0', amount=1)
8309 Row(product_id='CV140407A0-H2', parent_id='CV140407A0', amount=1)
8310 Row(product_id='CV140407A0-H3', parent_id='CV140407A0', amount=1)
8311 Row(product

9065 Row(product_id='003-2005-003-P-G', parent_id='003-2005-003-P', amount=1)
9066 Row(product_id='003-2005-003-P-G1', parent_id='003-2005-003-P', amount=1)
9067 Row(product_id='003-2005-003-P-H', parent_id='003-2005-003-P', amount=1)
9068 Row(product_id='003-2005-003-P-J', parent_id='003-2005-003-P', amount=2)
9069 Row(product_id='003-2005-003-P-J1', parent_id='003-2005-003-P', amount=2)
9070 Row(product_id='003-2005-003-P-K', parent_id='003-2005-003-P', amount=2)
9071 Row(product_id='003-2005-003-P-K1', parent_id='003-2005-003-P', amount=2)
9072 Row(product_id='003-2005-003-P-S', parent_id='003-2005-003-P', amount=1)
9073 Row(product_id='2090001068-A', parent_id='2090001068', amount=1)
9074 Row(product_id='2090001068-A1', parent_id='2090001068', amount=1)
9075 Row(product_id='2090001068-B', parent_id='2090001068', amount=1)
9076 Row(product_id='2090001068-B1', parent_id='2090001068', amount=1)
9077 Row(product_id='2090001068-B2', parent_id='2090001068', amount=1)
9078 Row(product_id=

10275 Row(product_id='8030206016-G3', parent_id='8030206016', amount=2)
10276 Row(product_id='8030206016-G4', parent_id='8030206016', amount=4)
10277 Row(product_id='8030206016-G5', parent_id='8030206016', amount=4)
10278 Row(product_id='8030206016-G6', parent_id='8030206016', amount=8)
10279 Row(product_id='8030206016-G7', parent_id='8030206016', amount=4)
10280 Row(product_id='8030206016-H3', parent_id='8030206016', amount=1)
10281 Row(product_id='8082506027-A', parent_id='8082506027', amount=1)
10282 Row(product_id='8082506027-A1', parent_id='8082506027', amount=1)
10283 Row(product_id='8082506027-B', parent_id='8082506027', amount=1)
10284 Row(product_id='8082506027-C', parent_id='8082506027', amount=1)
10285 Row(product_id='8082506027-D', parent_id='8082506027', amount=1)
10286 Row(product_id='8082506027-E', parent_id='8082506027', amount=1)
10287 Row(product_id='8082506027-E2', parent_id='8082506027', amount=1)
10288 Row(product_id='8082506027-F', parent_id='8082506027', amount=2

##### 2 用成2個Dataframe，再merge

In [10]:
#缺母件名稱的折床報表
df_parts_log = pd.DataFrame(real_parts_log)
df_parts_log.columns=['機台','子件料號','週數','週總耗時', '起始時間', '結束時間', '耗時','完成數量', '失敗數量']

#從bom去撈母件名稱
df_parent = pd.DataFrame(data_bom)
df_parent.columns=['子件料號','母件料號','數量']

In [11]:
data=pd.merge(df_parts_log, df_parent, how='left')  
#直接合起來最快，有些顯示NAN，是因為這個子件根本就沒有折床工序卻莫名奇妙地出現了，
#有的是新子件

data

,機台,子件料號,週數,週總耗時,起始時間,結束時間,耗時,完成數量,失敗數量,母件料號,數量
0,HG8025-1,1SLCD2007A0-A,23,7 days 14:57:18,2020-06-01 08:14:45,2020-06-01 08:29:36,00:14:51,2,0,1SLCD2007A0,1.0
1,HG1003ATC-01,1AK1D4004A1-A,23,7 days 14:57:18,2020-06-01 08:15:42,2020-06-01 08:21:12,00:05:30,1,0,1AK1D4004A1,1.0
2,HG1003ATC-01,42Z5300110-V,23,7 days 14:57:18,2020-06-01 08:21:14,2020-06-01 08:22:29,00:01:15,2,0,42Z5300110,1.0
3,HG1003ATC-01,PV116FU0114-C,23,7 days 14:57:18,2020-06-01 08:22:31,2020-06-01 08:25:34,00:03:03,3,0,PV116FU0114,1.0
4,HDS1303-3,1M42F1005C4-B,23,7 days 14:57:18,2020-06-01 08:24:15,2020-06-01 08:24:33,00:00:18,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17796,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:38:07,2020-08-01 16:39:52,00:01:45,2,0,PV860EL0002,1.0
17797,HG8025-1,PV860EL0001-A,31,9 days 18:32:06,2020-08-01 16:39:55,2020-08-01 16:55:53,00:15:58,2,0,PV860EL0001,1.0
17798,HDS1303-3,PV110FU0601-B,31,9 days 18:32:06,2020-08-01 16:52:32,2020-08-01 16:56:51,00:04:19,3,0,PV110FU0601,1.0
17799,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:55:56,2020-08-01 17:03:37,00:07:41,3,0,PV860EL0002,1.0


##### 3 找不到相對應的母件，根據它本身的子件名稱做切分，並加上異常值*記號

In [12]:
count=0
for index, row in data.iterrows():
    if type(row[9])==float:
        
        if len(row[1].split('-'))>=2:
            parent=[index,row[1].split('-')[0]]
            data.iloc[parent[0],9]='*'+parent[1]
            
        else:
            parent=[index,row[1]]
            data.iloc[parent[0],9]='*'+parent[1]
            
    if row[7]==0:  #這個地方處理的是'如果完成數量為0',前面用的「數量」就代表它的完成數量，如果也是0的話就只能是1了
        
        if row[10]>=1:
            print(row[10])
            data.iloc[index,7]=row[10]
            count+=1
            
        else:
            print(row[10])
            data.iloc[index,7]=1
            count+=1
count

nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
nan
2.0
nan
1.0
1.0
6.0
1.0
2.0
1.0
1.0
1.0
4.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
nan
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
nan
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
2.0
1.0
1.0
12.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
3.0
2.0
2.0
1.0
1.0
nan
nan
nan
nan
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
nan
nan
2.0
2.0
1.0
1.0
1.0
nan
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
nan
nan
1.0
nan
nan
2.0
nan
1.0
1.0
1.0
1.0
4.0
1.0
1.0
1.0
2.0
nan
1.0
nan
1.0
1.0
1.0
2.0
1.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
nan
2.0
2.0
2.0
2.0
1.0
1.0
1.0

1999

In [13]:
data

,機台,子件料號,週數,週總耗時,起始時間,結束時間,耗時,完成數量,失敗數量,母件料號,數量
0,HG8025-1,1SLCD2007A0-A,23,7 days 14:57:18,2020-06-01 08:14:45,2020-06-01 08:29:36,00:14:51,2.0,0,1SLCD2007A0,1.0
1,HG1003ATC-01,1AK1D4004A1-A,23,7 days 14:57:18,2020-06-01 08:15:42,2020-06-01 08:21:12,00:05:30,1.0,0,1AK1D4004A1,1.0
2,HG1003ATC-01,42Z5300110-V,23,7 days 14:57:18,2020-06-01 08:21:14,2020-06-01 08:22:29,00:01:15,2.0,0,42Z5300110,1.0
3,HG1003ATC-01,PV116FU0114-C,23,7 days 14:57:18,2020-06-01 08:22:31,2020-06-01 08:25:34,00:03:03,3.0,0,PV116FU0114,1.0
4,HDS1303-3,1M42F1005C4-B,23,7 days 14:57:18,2020-06-01 08:24:15,2020-06-01 08:24:33,00:00:18,1.0,0,*1M42F1005C4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17796,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:38:07,2020-08-01 16:39:52,00:01:45,2.0,0,PV860EL0002,1.0
17797,HG8025-1,PV860EL0001-A,31,9 days 18:32:06,2020-08-01 16:39:55,2020-08-01 16:55:53,00:15:58,2.0,0,PV860EL0001,1.0
17798,HDS1303-3,PV110FU0601-B,31,9 days 18:32:06,2020-08-01 16:52:32,2020-08-01 16:56:51,00:04:19,3.0,0,PV110FU0601,1.0
17799,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:55:56,2020-08-01 17:03:37,00:07:41,3.0,0,PV860EL0002,1.0


## 薪資表

##### 1 匯入薪資表

In [18]:
wage_table = pd.read_excel('薪資統計FOR成本.xlsx')  #讀取薪資表
wage_table

,日期,星期,週數,雷射薪資,折床薪資,焊接薪資,噴漆薪資,組立薪資
0,2020-01-01,星期三,1,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,星期四,1,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,星期五,1,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,星期六,1,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,星期日,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
361,2020-12-27,星期日,53,NaN,NaN,NaN,NaN,NaN
362,2020-12-28,星期一,53,NaN,NaN,NaN,NaN,NaN
363,2020-12-29,星期二,53,NaN,NaN,NaN,NaN,NaN
364,2020-12-30,星期三,53,NaN,NaN,NaN,NaN,NaN


##### 2 把它groupby，再單獨變成一個dataframe

In [19]:
agg=wage_table.groupby('週數')['折床薪資'].agg(np.sum)
real_wage_table=pd.DataFrame(agg)
real_wage_table.reset_index(inplace=True)
real_wage_table

,週數,折床薪資
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
5,6,0.0
6,7,0.0
7,8,0.0
8,9,0.0
9,10,0.0


##### 3 再merge

In [20]:
bending=pd.merge(data,real_wage_table, how='left')  
bending

,機台,子件料號,週數,週總耗時,起始時間,結束時間,耗時,完成數量,失敗數量,母件料號,數量,折床薪資
0,HG8025-1,1SLCD2007A0-A,23,7 days 14:57:18,2020-06-01 08:14:45,2020-06-01 08:29:36,00:14:51,2.0,0,1SLCD2007A0,1.0,52041.0
1,HG1003ATC-01,1AK1D4004A1-A,23,7 days 14:57:18,2020-06-01 08:15:42,2020-06-01 08:21:12,00:05:30,1.0,0,1AK1D4004A1,1.0,52041.0
2,HG1003ATC-01,42Z5300110-V,23,7 days 14:57:18,2020-06-01 08:21:14,2020-06-01 08:22:29,00:01:15,2.0,0,42Z5300110,1.0,52041.0
3,HG1003ATC-01,PV116FU0114-C,23,7 days 14:57:18,2020-06-01 08:22:31,2020-06-01 08:25:34,00:03:03,3.0,0,PV116FU0114,1.0,52041.0
4,HDS1303-3,1M42F1005C4-B,23,7 days 14:57:18,2020-06-01 08:24:15,2020-06-01 08:24:33,00:00:18,1.0,0,*1M42F1005C4,NaN,52041.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17796,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:38:07,2020-08-01 16:39:52,00:01:45,2.0,0,PV860EL0002,1.0,50193.4
17797,HG8025-1,PV860EL0001-A,31,9 days 18:32:06,2020-08-01 16:39:55,2020-08-01 16:55:53,00:15:58,2.0,0,PV860EL0001,1.0,50193.4
17798,HDS1303-3,PV110FU0601-B,31,9 days 18:32:06,2020-08-01 16:52:32,2020-08-01 16:56:51,00:04:19,3.0,0,PV110FU0601,1.0,50193.4
17799,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:55:56,2020-08-01 17:03:37,00:07:41,3.0,0,PV860EL0002,1.0,50193.4


##### 4. 用加進來的薪資算出總成本

In [21]:
divided_cost=[]
for index ,row in bending.iterrows():
    divided_cost.append(row[11]*row[6]/row[3])
bending['總成本']=divided_cost
bending

,機台,子件料號,週數,週總耗時,起始時間,結束時間,耗時,完成數量,失敗數量,母件料號,數量,折床薪資,總成本
0,HG8025-1,1SLCD2007A0-A,23,7 days 14:57:18,2020-06-01 08:14:45,2020-06-01 08:29:36,00:14:51,2.0,0,1SLCD2007A0,1.0,52041.0,70.400631
1,HG1003ATC-01,1AK1D4004A1-A,23,7 days 14:57:18,2020-06-01 08:15:42,2020-06-01 08:21:12,00:05:30,1.0,0,1AK1D4004A1,1.0,52041.0,26.074308
2,HG1003ATC-01,42Z5300110-V,23,7 days 14:57:18,2020-06-01 08:21:14,2020-06-01 08:22:29,00:01:15,2.0,0,42Z5300110,1.0,52041.0,5.925979
3,HG1003ATC-01,PV116FU0114-C,23,7 days 14:57:18,2020-06-01 08:22:31,2020-06-01 08:25:34,00:03:03,3.0,0,PV116FU0114,1.0,52041.0,14.459389
4,HDS1303-3,1M42F1005C4-B,23,7 days 14:57:18,2020-06-01 08:24:15,2020-06-01 08:24:33,00:00:18,1.0,0,*1M42F1005C4,NaN,52041.0,1.422235
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17796,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:38:07,2020-08-01 16:39:52,00:01:45,2.0,0,PV860EL0002,1.0,50193.4,6.242029
17797,HG8025-1,PV860EL0001-A,31,9 days 18:32:06,2020-08-01 16:39:55,2020-08-01 16:55:53,00:15:58,2.0,0,PV860EL0001,1.0,50193.4,56.951080
17798,HDS1303-3,PV110FU0601-B,31,9 days 18:32:06,2020-08-01 16:52:32,2020-08-01 16:56:51,00:04:19,3.0,0,PV110FU0601,1.0,50193.4,15.397004
17799,HG8025-1,PV860EL0002-A,31,9 days 18:32:06,2020-08-01 16:55:56,2020-08-01 17:03:37,00:07:41,3.0,0,PV860EL0002,1.0,50193.4,27.405478


## 接下來就是算每個子件的單一折床成本

In [23]:
true_bending_cost=[]

grouped = bending.groupby(['子件料號','母件料號']) #用groupby的方法處理
agg = grouped['完成數量','總成本'].agg(np.sum)
bending_cost = pd.DataFrame(agg)
bending_cost.reset_index(inplace=True)

for i,q in zip(bending_cost['總成本'],bending_cost['完成數量']):
    true_bending_cost.append(i/q)
    
bending_cost['子件折床單一成本']=true_bending_cost
bending_cost

,子件料號,母件料號,完成數量,總成本,子件折床單一成本
0,,*,2.0,34.658120,17.329060
1,0032005003-B3,*0032005003,1.0,48.033302,48.033302
2,0032005003G-A,*0032005003G,61.0,412.472769,6.761849
3,0032005003G-A1,*0032005003G,60.0,831.260402,13.854340
4,0032005003G-B,*0032005003G,30.0,454.178163,15.139272
...,...,...,...,...,...
8171,X8SQ349A-A,X8SQ349A,2.0,72.692010,36.346005
8172,X8SQ349A-B,X8SQ349A,4.0,27.022465,6.755616
8173,X8SQ349A-E,X8SQ349A,6.0,13.827284,2.304547
8174,X8SQ386A-B,X8SQ386A,2.0,5.214862,2.607431


In [24]:
bending_cost.to_csv('bending0531-0801.csv', index=False, mode='w', header=True, encoding='utf_8_sig')